In [ ]:
import pickle


In [120]:
mse_tune = []
mse = []

with open('./gleb/mse_after.txt') as f:
    for i in range(88):
        mse_tune.append(float(f.readline()))

with open('./gleb/mse_before.txt') as f:
    for i in range(88):
        mse.append(float(f.readline()))

In [102]:
# with open('./gleb/all_preds.txt', 'rb') as f:
#     f.readline
    
with open('./gleb/mse_after', 'rb') as f:
    mse_after = pickle.load(f)

with open('./gleb/mse_before', 'rb') as f:
    mse_before = pickle.load(f)

UnpicklingError: invalid load key, '['.

In [20]:
sample_sub_1 = pd.read_csv('./gleb/data/sample_sub_1.csv')
sample_sub_1

,year,station_id,month,day,date,delta_stage_max
0,1993,3019,4,111,1993-04-21,0
1,1993,3019,4,112,1993-04-22,0
2,1993,3019,4,113,1993-04-23,0
3,1993,3019,4,114,1993-04-24,0
4,1993,3019,4,115,1993-04-25,0
...,...,...,...,...,...,...
611,2013,3230,4,120,2013-04-30,0
612,2013,3230,5,121,2013-05-01,0
613,2013,3230,5,122,2013-05-02,0
614,2013,3230,5,123,2013-05-03,0


In [100]:
def create_submission(preds, mse, mse_tune):
    df = pd.DataFrame(columns=['year', 'station_id', 'month', 'day', 'date', 'delta_stage_max'])
    
    for index, row in enumerate(preds):
        new_row = {}
        date = pd.to_datetime(row[0])
        station_id = row[1]
        
        new_row['station_id'] = station_id
        new_row['year'] = date.year
        new_row['month'] = date.month
        new_row['day'] = date.timetuple().tm_yday
        new_row['date'] = date
        
        if mse_tune[index] < mse[index]:
            predictions = row[3]
        else:
            predictions = row[2]
            
        for i, val in enumerate(predictions):
            new_row['stage_max'] = val
            new_row['day'] = (date + datetime.timedelta(days=i)).timetuple().tm_yday 
            
            df = df.append(new_row, ignore_index=True)
            
    
            
    return df
            

In [132]:
results = create_submission(array, mse, mse_tune)
results['delta_stage_max'] = results['stage_max'] - results['stage_max'].shift(1)
results = results.backfill()
results = results.iloc[:,:-1]
results.head()

In [158]:
results

,year,station_id,month,day,date,delta_stage_max
0,1993,3019,4,111,1993-04-21,0.274274
1,1993,3019,4,112,1993-04-21,0.274274
2,1993,3019,4,113,1993-04-21,0.535656
3,1993,3019,4,114,1993-04-21,-0.568193
4,1993,3019,4,115,1993-04-21,1.107205
...,...,...,...,...,...,...
611,2013,3230,4,120,2013-04-28,-11.225376
612,2013,3230,4,121,2013-04-28,-12.410070
613,2013,3230,4,122,2013-04-28,-12.306396
614,2013,3230,4,123,2013-04-28,-9.335737


In [156]:
sample_sub_1.to_csv('submission.csv', index=False)

In [157]:
pd.read_csv('submission.csv')

,year,station_id,month,day,date,delta_stage_max
0,1993,3019,4,111,1993-04-21,0.274274
1,1993,3019,4,112,1993-04-22,0.274274
2,1993,3019,4,113,1993-04-23,0.535656
3,1993,3019,4,114,1993-04-24,-0.568193
4,1993,3019,4,115,1993-04-25,1.107205
...,...,...,...,...,...,...
611,2013,3230,4,120,2013-04-30,-11.225376
612,2013,3230,5,121,2013-05-01,-12.410070
613,2013,3230,5,122,2013-05-02,-12.306396
614,2013,3230,5,123,2013-05-03,-9.335737


In [155]:
sample_sub_1['delta_stage_max'] = results['delta_stage_max']
sample_sub_1

,year,station_id,month,day,date,delta_stage_max
0,1993,3019,4,111,1993-04-21,0.274274
1,1993,3019,4,112,1993-04-22,0.274274
2,1993,3019,4,113,1993-04-23,0.535656
3,1993,3019,4,114,1993-04-24,-0.568193
4,1993,3019,4,115,1993-04-25,1.107205
...,...,...,...,...,...,...
611,2013,3230,4,120,2013-04-30,-11.225376
612,2013,3230,5,121,2013-05-01,-12.410070
613,2013,3230,5,122,2013-05-02,-12.306396
614,2013,3230,5,123,2013-05-03,-9.335737
